## Import Libraries

In [1]:
import matplotlib.pyplot as plt
from numpy import sin, cos, sqrt, pi, linspace, arange, deg2rad, rad2deg, array, arcsin, arccos,sort, argsort, argwhere, argmin, argmax, interp, concatenate, zeros
from scipy.spatial import distance
from numpy import linalg as LA
import warnings
warnings.simplefilter('ignore')

def sec(x):
    return 1/cos(x)
def tan(x):
    return sin(x)/cos(x)

## Initialize Parameters 

In [2]:
n      = 1.33
c      = 299792458/n
h      = 4.5
Dia    = 7.3   #--------------------- diameter of tank
R      = 7.3/2 #--------------------- radius of tank
v      = n * c
times  = linspace(1e-12,1e-6,30000)

#x1,y1,z1 = [1.0 , sqrt(R**2-1.0**2),1.3] #----------------------------------------- Coordinates of particle at top tanker lid      #------PARAM
#x2,y2,z2 = [-2.0, sqrt(R**2-2.0**2),1.3] #-------------------- Calculate bottom y coordinate
x1,y1,z1 = [R,0,4.2]
x2,y2,z2 = [-R,0,4.2]
A      = array([x1,y1,z1])
B      = array([x2,y2,z2])
D1     = array([0, 0, 0])                                #------ Central    PMT number 1 
D2     = array([1.85*cos(2*pi/3) , 1.85*sin(2*pi/3), 0]) #------ Non-Radial PMT number 2
D3     = array([1.85*cos(4*pi/3) , 1.85*sin(4*pi/3), 0]) #------ Non-Radial PMT number 3
D4     = array([1.85*cos(0)      , 1.85*sin(0     ), 0]) #------ Radial     PMT number 4

nAB    = LA.norm(B-A)
nBD1   = LA.norm(B-D1)
nBD2   = LA.norm(B-D2)
nBD3   = LA.norm(B-D3)
nBD4   = LA.norm(B-D4)

limit     = nAB 
print('A   : ', A)
print('B   : ', B)
print('D1  : ', D1)
print('D2  : ', D2)
print('D3  : ', D3)
print('D4  : ', D4)
print('AB  : ', nAB)
print('BD1 :  %.2f m\nBD2 :  %.2f m\nBD3 :  %.2f m\nBD4 :  %.2f m'%(nBD1,nBD2,nBD3,nBD4) )
print('Upper limit: ', limit, "m")

A   :  [3.65 0.   4.2 ]
B   :  [-3.65  0.    4.2 ]
D1  :  [0 0 0]
D2  :  [-0.925     1.602147  0.      ]
D3  :  [-0.925    -1.602147  0.      ]
D4  :  [1.85 0.   0.  ]
AB  :  7.3
BD1 :  5.56 m
BD2 :  5.26 m
BD3 :  5.26 m
BD4 :  6.92 m
Upper limit:  7.3 m


## Computations 

In [3]:
def entry_brightness(nAD,ang1,c,v,den,A,B,D):
    x         = 0
    t1        = 0
    k         = sqrt(x**2 + nAD**2 - 2*x*nAD*cos(ang1))
    t2        = k/c
    tt        = t1 + t2
    xtermA    = c*c*tt*v - nAD*v*v*cos(ang1) 
    xtermB    = v*v*( -v*v*nAD**2 + c*c*(nAD**2 + tt*tt*v*v) - 2*c*c*nAD*tt*v*cos(ang1) + (nAD*v*cos(ang1))**2)
    vtermA    = c*c*v**3*(tt*v - nAD*cos(ang1))
    xa, xb    = (xtermA + sqrt(xtermB))/den , (xtermA - sqrt(xtermB))/den
    va, vb    = (c*c*v + vtermA/sqrt(xtermB))/den, (c*c*v - vtermA/sqrt(xtermB))/den

    r, ra, rb = x/nAB, xa/nAB, xb/nAB
    X        = array([ (1-r) *A[0] + r *B[0] , (1-r) *A[1] + r *B[1] , (1-r) *A[2]+r *B[2] ])
    Xa       = array([ (1-ra)*A[0] + ra*B[0] , (1-ra)*A[1] + ra*B[1] , (1-ra)*A[2]+ra*B[2] ])
    Xb       = array([ (1-rb)*A[0] + rb*B[0] , (1-rb)*A[1] + rb*B[1] , (1-rb)*A[2]+rb*B[2] ])

    ka, kb   = LA.norm(D-Xa) , LA.norm(D-Xb)
    phia     = arccos ( sum( (D-Xa)*(D-A) ) / (LA.norm(D-Xa)*LA.norm(D-A)) )
    phib     = arccos ( sum( (D-Xb)*(D-B) ) / (LA.norm(D-Xb)*LA.norm(D-B)) )
    alphaa   = arccos ( sum( (D-Xa)*(B-Xa)) / (LA.norm(D-Xa)*LA.norm(B-Xa)) )
    alphab   = arccos ( sum( (D-Xb)*(B-Xb)) / (LA.norm(D-Xb)*LA.norm(B-Xb)) )

    vta      = va * sin(alphaa)
    vtb      = vb * sin(alphab)
    omegaa   = (vta / ka)
    omegab   = (vtb / kb)
    ba       = abs(omegaa / (ka**2))
    bb       = abs(omegab / (kb**2))

    return ba

def A_t_vs_phi(x,y,color,mylabel,PHI):
    plt.plot(x, y, c=color, ls='-' , label=mylabel, lw=2)
def B_t_vs_phi(x,y,color,mylabel,PHI):
    plt.plot(x, y, c=color, ls='-' , label=mylabel, lw=2)
def both_t_vs_phi(xa,ya,xb,yb,color,mylabel,PHI):
    plt.plot(xa, ya, c=color, ls='--' , label=mylabel, lw=2)
    plt.plot(xb, yb, c=color, ls='-'  , label=mylabel, lw=2)
    #if detector!=4:
        #plt.axhline(phi, c='k', ls='--')
        #if detector==2:
        #    plt.text(28, phi-1, r'$\theta_C = %.2f ^{\circ}$'%phi , fontsize=15)
        #plt.text(28, phi+1, r'$\theta_C = %.2f ^{\circ}$'%phi , fontsize=15)       
    #plt.axvline(0, c='k', ls=':')


def both_phi_vs_t(tp,phip,tm,phim,color,mylabel,phi):
    plt.plot(tp, phip, c=color, ls='-' , label=mylabel,lw=2)
    plt.plot(tm, phim, c=color, ls='-.',lw=2)
    if detector!=4:
        plt.axhline(phi, c='k', ls='--')
        if detector==2:
            plt.text(28, phi-4, r'$\theta_C = %.2f ^{\circ}$'%phi , fontsize=15)
        else:
            plt.text(28, phi+1, r'$\theta_C = %.2f ^{\circ}$'%phi , fontsize=15)
    
    plt.axvline(0, c='k', ls=':')
    

def plus_b_vs_t(t,bp,color,mylabel):  # Use times  = linspace(1e-12,1e-7,2000000)
    plt.plot(t, bp, c=color, ls='-', label=mylabel, lw=2.5)
    plt.axhline(1, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')

def minus_b_vs_t(t,bm,color,mylabel):  # Use times  = linspace(1e-12,1e-7,2000000)
    plt.plot(t, bm, c=color, ls='--',label=mylabel, lw=2.5)
    plt.axhline(1, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')

def both_b_vs_t(tp,bp,tm,bm,color,mylabel):  # Use times  = linspace(1e-12,1e-7,2000000)
    plt.plot(tp, bp, c=color, ls='-', label=mylabel, lw=2.5)
    plt.plot(tm, bm, c=color, ls='--', lw=2.5)
    plt.axhline(1, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')

    
def plus_b_vs_phi(phip,bp,color,mylabel,phi):
    plt.plot(phip, bp, c=color, ls='-', label=mylabel, lw=2.5)
    plt.axhline(1,  c='k', ls=':')

def minus_b_vs_phi(phim,bm,color,mylabel,phi):
    plt.plot(phim, bm, c=color, ls='--',label=mylabel, lw=2.5)
    plt.axhline(1,  c='k', ls=':')

def both_b_vs_phi(phip,bp,phim,bm,color,mylabel,phi):
    plt.plot(phip, bp, c=color, ls='-', label=mylabel, lw=2.5)
    plt.plot(phim, bm, c=color, ls='--', lw=2.5)
    if detector!=4:
        plt.axvline(phi,c='k', ls=':')
        if detector==3:
            plt.text(phi+0.5, 500, r'$\theta_C = %.2f ^{\circ}$'%phi , fontsize=15)
        elif detector==1:
            plt.text(phi+0.5, 1.2, r'$\theta_C = %.2f ^{\circ}$'%phi , fontsize=15)
        else:
            plt.text(phi-10, 2000, r'$\theta_C = %.2f ^{\circ}$'%phi , fontsize=15)
    plt.axhline(1, c='k', ls=':')
    
    
def A_x_vs_t(ta,xa,color,mylabel,xc):
    plt.plot(ta,xa, c=color, ls='-' , label=mylabel, lw=2.5)
    if (0<=xc<=limit):
        plt.text(30, xc+0.05, r'$z_C = %.2f $'%(round(xc,2)) , fontsize=15)
        plt.axhline(xc, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')

def toB_x_vs_t(tb,xb,color,mylabel,xc):
    plt.plot(tb, xb, c=color, ls='-.' , label=mylabel, lw=2.5)
    if (0<=xc<=limit):
        plt.text(30, xc+0.05, r'$z_C = %.2f $'%(round(xc,2)) , fontsize=15)
        plt.axhline(xc, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')

def both_x_vs_t(ta,xa,tb,xb,color,mylabel,xc):
    plt.plot(ta, xa, c=color, ls='-' , label=mylabel, lw=2.5)
    plt.plot(tb, xb, c=color, ls='-.', lw=2.5)
    if (0<=xc<=limit):
        if detector==3:
            plt.text(12, 2.1, r'$z_C = %.2f $'%(round(xc,2)) , fontsize=15)
        elif detector==1:
            plt.text(13, xc+0.05, r'$z_C = %.2f $'%(round(xc,2)) , fontsize=15)     
        else:
            plt.text(30, xc+0.05, r'$z_C = %.2f $'%(round(xc,2)) , fontsize=15)
        plt.axhline(xc, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')


In [11]:
%matplotlib
plt.figure(figsize=(15,12))

#plotme      = 'b vs phi'
#plotme      = 'b vs t'
plotme      = 't vs phi'
#plotme      = 'x vs t'

detectors   = [1,2,3,4]
coordinates = [D1,D2,D3,D4]
colors      = ['k','r','b','g']
labels      = ['Central Detector', "Detector 2",'Detector 3','Detector 4']

for detector, D, color, mylabel in zip(detectors, coordinates, colors, labels):
    print("Detector ", detector)
    ang1   = sum((A-D)*(A-B)) / (LA.norm(A-D)*LA.norm(A-B))
    ang2   = sum((B-D)*(A-B)) / (LA.norm(B-D)*LA.norm(A-B))
    den    = c*c-v*v
    nAD    = LA.norm(A-D)
    xc     = nAD*cos(ang1) + c*nAD*sin(ang1)*sqrt(-den)/den
    T, XtA, XtB, BtA, BtB, PHItA, PHItB = [], [], [], [], [], [], []
    
    nAD = LA.norm(A-D)
    for iii,t in enumerate(times):
        x         = v*t
        t1        = t
        k         = sqrt(x**2 + nAD**2 - 2*x*nAD*cos(ang1))
        t2        = k/c
        tt        = t1 + t2
        xtermA    = c*c*tt*v - nAD*v*v*cos(ang1) 
        xtermB    = v*v*( -v*v*nAD**2 + c*c*(nAD**2 + tt*tt*v*v) - 2*c*c*nAD*tt*v*cos(ang1) + (nAD*v*cos(ang1))**2)
        vtermA    = c*c*v**3*(tt*v - nAD*cos(ang1))
        xa, xb    = (xtermA + sqrt(xtermB))/den , (xtermA - sqrt(xtermB))/den
        va, vb    = (c*c*v + vtermA/sqrt(xtermB))/den, (c*c*v - vtermA/sqrt(xtermB))/den

        r, ra, rb = x/nAB, xa/nAB, xb/nAB
        X        = array([ (1-r) *A[0] + r *B[0] , (1-r) *A[1] + r *B[1] , (1-r) *A[2]+r *B[2] ])
        Xa       = array([ (1-ra)*A[0] + ra*B[0] , (1-ra)*A[1] + ra*B[1] , (1-ra)*A[2]+ra*B[2] ])
        Xb       = array([ (1-rb)*A[0] + rb*B[0] , (1-rb)*A[1] + rb*B[1] , (1-rb)*A[2]+rb*B[2] ])
        
        ka, kb   = LA.norm(D-Xa) , LA.norm(D-Xb)
        phia     = arccos ( sum( (D-Xa)*(D-A) ) / (LA.norm(D-Xa)*LA.norm(D-A)) )
        phib     = arccos ( sum( (D-Xb)*(D-B) ) / (LA.norm(D-Xb)*LA.norm(D-B)) )
        alphaa   = arccos ( sum( (D-Xa)*(B-Xa)) / (LA.norm(D-Xa)*LA.norm(B-Xa)) )
        alphab   = arccos ( sum( (D-Xb)*(B-Xb)) / (LA.norm(D-Xb)*LA.norm(B-Xb)) )
        
        vta      = va * sin(alphaa)
        vtb      = vb * sin(alphab)
        omegaa   = (vta / ka)
        omegab   = (vtb / kb)
        ba       = abs(omegaa / (ka**2))
        bb       = abs(omegab / (kb**2))
        
        XtA.append(xa)
        XtB.append(xb)
        BtA.append(ba)
        BtB.append(bb)
        PHItA.append(rad2deg(phia))
        PHItB.append(rad2deg(phib))
        T.append(tt*1e9)
        
    rc  = xc/nAB
    Xc  = array([ (1-rc)*A[0] + rc*B[0] , (1-rc)*A[1] + rc*B[1] , (1-rc)*A[2]+rc*B[2] ])
    PHI = arccos( sum((D-Xc)*(D-A))  / (LA.norm(D-Xc)*LA.norm(D-A)) ) ; PHI = rad2deg(PHI)

    print("PHI: ",round(rad2deg(PHI),2), "XC: ", round(xc,2))
    XtA, XtB, TT, BtA, BtB, PHItA, PHItB = array(XtA), array(XtB), array(T), array(BtA), array(BtB), array(PHItA), array(PHItB)
        
    cona = [(XtA>=0) & (XtA<=limit)]
    conb = [(XtB>=0) & (XtB<=limit)]
    alen = len (XtA[cona])
    blen = len (XtB[conb])
    
    if detector==1:
        den_bright = entry_brightness(nAD,ang1,c,v,den,A,B,D)
    
    if alen == 0:
        if blen == 0:
            print("Images outside tank. Skipping...")
        else:
            print('Only image moving forward towards B. Processing...')
            TTb, XtB, BtB, PHItB = TT[conb], XtB[conb], BtB[conb]/den_bright, PHItB[conb]
            if   plotme=='b vs phi': B_b_vs_phi (x=PHItB, y=BtB  , color=color, mylabel=mylabel, PHI=PHI)
            elif plotme=='t vs b'  : B_t_vs_b   (x=TTb  , y=BtB  , color=color, mylabel=mylabel         )
            elif plotme=='t vs phi': B_t_vs_phi (x=TTb  , y=PHItB, color=color, mylabel=mylabel, PHI=PHI)
            elif plotme=='t vs x'  : B_t_vs_x   (x=TTb  , y=XtB  , color=color, mylabel=mylabel, xc =xc )
    
    elif alen != 0:
        if blen == 0:
            print('Only image moving back towards A. Processing...')
            TTa, XtA, BtA, PHItA = TT[cona], XtA[cona], BtA[cona]/den_bright, PHItA[cona]
            if   plotme=='b vs phi': A_b_vs_phi (x=PHItA, y=BtA  , color=color, mylabel=mylabel, PHI=PHI)
            elif plotme=='t vs b'  : A_t_vs_b   (x=TTa  , y=BtA  , color=color, mylabel=mylabel         )
            elif plotme=='t vs phi': A_t_vs_phi (x=TTa  , y=PHItA, color=color, mylabel=mylabel, PHI=PHI)
            elif plotme=='t vs x'  : A_t_vs_x   (x=TTa  , y=XtA  , color=color, mylabel=mylabel, xc =xc )
        else:
            print('Both images moving in both directions. Processing...')
            TTa,XtA,BtA,PHItA = TT[cona],XtA[cona],BtA[cona]/den_bright,PHItA[cona]
            TTb,XtB,BtB,PHItB = TT[conb],XtB[conb],BtB[conb]/den_bright,PHItB[conb]
            if   plotme=='b vs phi': both_b_vs_phi (xa=PHItA, ya=BtA  , xb=PHItB, yb=BtB  , color=color, mylabel=mylabel, PHI=PHI)
            elif plotme=='t vs b'  : both_t_vs_b   (xa=TTa  , ya=BtA  , xb=TTb  , yb=BtB  , color=color, mylabel=mylabel     )
            elif plotme=='t vs phi': both_t_vs_phi (xa=TTa  , ya=PHItA, xb=TTb  , yb=PHItB, color=color, mylabel=mylabel, PHI=PHI)
            elif plotme=='t vs x'  : both_t_vs_x   (xa=TTa  , ya=XtA  , xb=TTb  , yb=XtB  , color=color, mylabel=mylabel, xc=xc  )
    plt.axhline(PHI)

plt.legend()#loc='upper right', prop={'size': 14})
plt.tick_params(axis='both', direction='in', labelsize=18)

#put stuff here from the next cell
plt.show()

Using matplotlib backend: Qt5Agg
Detector  1
PHI:  255.89 XC:  0.54
Both images moving in both directions. Processing...
Detector  2
PHI:  23.15 XC:  0.06
Both images moving in both directions. Processing...
Detector  3
PHI:  23.15 XC:  0.06
Both images moving in both directions. Processing...
Detector  4
PHI:  1655.92 XC:  2.22
Both images moving in both directions. Processing...


In [ ]:
alen

In [ ]:
# phi vs t
plt.xlabel('time since muon entry (ns)',fontsize=18)
plt.ylabel(r'angular locations $\phi_{pm}\;(in\;degrees)$', fontsize=18)
plt.xticks(arange(10,37,2))
plt.xlim(10.02,36.07)

# b vs t
plt.xlabel('time since muon entry (ns)',fontsize=18)
plt.ylabel('relative brightness ($b/b_{entry}$)', fontsize=18) 
plt.xticks(arange(11,38,2))
plt.yscale('log')
plt.xlim(10.02,36.07)
plt.ylim(0.34,286.5)

# b vs phi
plt.xlabel(r'angular locations $\phi_{pm}\;(in\;degrees)$',fontsize=18)
plt.ylabel(r'relative brightness ($b/b_{entry}$)', fontsize=18) 
plt.yscale('log')

# z vs t
plt.xlabel('time since muon entry (ns)',fontsize=18)
plt.ylabel(r'image heights $z_{pm} (in\;meters)$'     , fontsize=18).set_rotation(90)
plt.xticks(arange(10,38,2))
plt.ylim(-0.02,limit)
plt.xlim(9.99,36.07)